In [1]:
import pandas as pd

In [10]:
G = {
    'A': {('B', 1)},
    'B': {('A', 1), ('C', 1)},
    'C': {('B', 1), ('D', 1), ('E', 1)},
    'D': {('C', 1), ('E', 1), ('I', 1)},
    'E': {('C', 1), ('D', 1), ('F', 1), ('H', 1)},
    'F': {('E', 1), ('G', 1), ('R', 1)},
    'G': {('F', 1), ('H', 1), ('T', 1)},
    'H': {('E', 1), ('G', 1), ('J', 1)},
    'I': {('D', 1)},
    'J': {('H', 1), ('K', 1), ('L', 1)},
    'K': {('J', 1), ('L', 1)},
    'L': {('J', 1), ('K', 1), ('M', 1)},
    'M': {('L', 1), ('O', 1), ('R', 1), ('T', 1)},
    'N': {('O', 1), ('P', 1), ('Q', 1), ('T', 1)},
    'O': {('M', 1), ('N', 1), ('P', 1)},
    'P': {('N', 1), ('O', 1), ('Q', 1), ('R', 1)},
    'Q': {('N', 1), ('P', 1), ('R', 1), ('S', 1), ('U', 1)},
    'R': {('F', 1), ('M', 1), ('P', 1), ('Q', 1), ('S', 1)},
    'S': {('Q', 1), ('R', 1), ('T', 1), ('U', 1)},
    'T': {('G', 1), ('M', 1), ('N', 1), ('S', 1), ('U', 1)},
    'U': {('Q', 1), ('S', 1), ('T', 1)}}

G = {
    'A': {('C', 1), ('D', 1), ('E', 1), ('F', 1)},
    'B': {('C', 1), ('E', 1), ('H', 1)},
    'C': {('A', 1), ('B', 1), ('E', 1), ('F', 1), ('G', 1)},
    'D': {('A', 1), ('H', 1)},
    'E': {('A', 1), ('B', 1), ('C', 1), ('K', 1)},
    'F': {('A', 1), ('C', 1), ('H', 1), ('L', 1)},
    'G': {('C', 1), ('H', 1), ('L', 1)},
    'H': {('B', 1), ('D', 1), ('F', 1), ('G', 1)},
    'I': {('J', 1), ('K', 1), ('L', 1), ('O', 1), ('P', 1)},
    'J': {('I', 1), ('M', 1), ('O', 1)},
    'K': {('E', 1), ('I', 1), ('L', 1), ('M', 1), ('N', 1), ('O', 1)},
    'L': {('F', 1), ('G', 1), ('I', 1), ('K', 1), ('N', 1)},
    'M': {('J', 1), ('K', 1)},
    'N': {('K', 1), ('L', 1)},
    'O': {('I', 1), ('J', 1), ('K', 1)},
    'P': {('I', 1)}}

In [16]:
def Louvain(graphe, gamma=1):
    
    if len(graphe) == 1:
        return graphe
    
    def gain_modularite(sommet, voisin, graphe, clusters, correspondance, gamma):
        
        sin = 0
        for s1 in clusters[correspondance[voisin]]:
            sin += sum(list(arete[1] for arete in graphe[s1] if arete[0] in clusters[correspondance[voisin]]))
        sin = sin/2
        
        stot = 0
        for s1 in set(graphe.keys()).difference(clusters[correspondance[voisin]]):
            stot += sum(list(arete[1] for arete in graphe[s1] if arete[0] in clusters[correspondance[voisin]]))
        stot += sin
            
        ki = sum(list(arete[1] for arete in graphe[sommet]))
        ki_in = sum(list(arete[1] for arete in graphe[sommet] if arete[0] in clusters[correspondance[voisin]]))
        
        m = 0
        for s1 in graphe:
            m += sum(list(arete[1] for arete in graphe[s1]))
        m /= 2
        
        p1 = (ki_in/(2*m))
        p2 = gamma*((stot*ki)/(2*(m**2)))
        
        return p1-p2
    
    def contraction(sommet, voisin, graphe):

        sf = f"{sommet}/{voisin}"
        nouveau_graphe = graphe.copy()

        for s in graphe:
            nouveau_graphe[s] = {arete for arete in graphe[s] if arete[0] not in {sommet, voisin}}
        del nouveau_graphe[sommet], nouveau_graphe[voisin]

        for s in nouveau_graphe:
            nouveau_graphe[s] = nouveau_graphe[s].union({(sf, sum(list(map(lambda arete: arete[1], {arete for arete in graphe[s] if arete[0] in {sommet, voisin}}))))})
            nouveau_graphe[s] = {(v, poids) for v, poids in nouveau_graphe[s] if poids > 0}

        nouveau_graphe[sf] = set()    

        for s in nouveau_graphe:
            nouveau_graphe[sf] = nouveau_graphe[sf].union({(s, arete[1]) for arete in nouveau_graphe[s] if arete[0] == sf})

        nouveau_graphe[sf] = nouveau_graphe[sf].union(({(sf, sum(list(map(lambda arete: arete[1], {arete for arete in graphe[sommet].union(graphe[voisin]) if arete[0] in {sommet, voisin}}))))}))
        nouveau_graphe[sf] = {(v, poids) for v, poids in nouveau_graphe[sf] if poids > 0}

        return nouveau_graphe
        
    correspondance = {sommet: indice for indice, sommet in enumerate(graphe.keys())}
    clusters = {indice: {sommet} for indice, sommet in enumerate(graphe.keys())}
    
    changement = True
    precedent = -1
    cpt = 0
    
    while changement:
        
        changement = False
    
        for sommet in graphe:
            max_local, attracteur = -1, -1
            for arete in graphe[sommet]:
                
                if arete[0] != sommet:

                    delta_q = gain_modularite(sommet, arete[0], graphe, clusters, correspondance, gamma)

                    if delta_q > max_local:
                        max_local, attracteur = delta_q, arete[0]
                    
            if max_local > 1e-07 and correspondance[sommet] != correspondance[attracteur] and precedent != sommet:
                cpt += 1
                changement = True
                precedent = sommet
                clusters[correspondance[sommet]] = clusters[correspondance[sommet]].difference({sommet})
                correspondance[sommet] = correspondance[attracteur]
                clusters[correspondance[sommet]] = clusters[correspondance[attracteur]].union({sommet})
                
    if cpt == 0:
        return graphe
                  
    clusters = {indice: groupe for indice, groupe in clusters.items() if groupe != set()}
    
    for cluster in clusters.values():
        init = list(cluster)[0]
        cluster = cluster.difference({init})
        while cluster != set():
            suivant = list(cluster)[0]
            graphe = contraction(init, suivant, graphe)
            cluster = cluster.difference({suivant})
            init += f"/{suivant}"
            
    return Louvain(graphe, gamma)
    

In [17]:
Louvain(G)

{'B/A/E/C': {('B/A/E/C', 10), ('D/G/H', 3), ('M/P/O/J/I/K', 1), ('N/F/L', 2)},
 'D/G/H': {('B/A/E/C', 3), ('D/G/H', 4), ('N/F/L', 2)},
 'M/P/O/J/I/K': {('B/A/E/C', 1), ('M/P/O/J/I/K', 16), ('N/F/L', 3)},
 'N/F/L': {('B/A/E/C', 2), ('D/G/H', 2), ('M/P/O/J/I/K', 3), ('N/F/L', 4)}}

In [18]:
Louvain(G, 0.8)

{'M/P/L/I/O/N/J/K': {('D/G/H/B/A/E/F/C', 3), ('M/P/L/I/O/N/J/K', 24)},
 'D/G/H/B/A/E/F/C': {('D/G/H/B/A/E/F/C', 22), ('M/P/L/I/O/N/J/K', 3)}}

In [19]:
Louvain(G, 1.8)

{'F': {('A/D', 1), ('B/E/C', 1), ('G/H', 1), ('N/L', 1)},
 'A/D': {('A/D', 2), ('B/E/C', 2), ('F', 1), ('G/H', 1)},
 'B/E/C': {('A/D', 2), ('B/E/C', 4), ('F', 1), ('G/H', 2), ('M/K', 1)},
 'G/H': {('A/D', 1), ('B/E/C', 2), ('F', 1), ('G/H', 2), ('N/L', 1)},
 'N/L': {('F', 1), ('G/H', 1), ('M/K', 2), ('N/L', 2), ('O/P/I/J', 1)},
 'O/P/I/J': {('M/K', 3), ('N/L', 1), ('O/P/I/J', 8)},
 'M/K': {('B/E/C', 1), ('M/K', 2), ('N/L', 2), ('O/P/I/J', 3)}}

In [20]:
interactome = pd.read_csv("Saccharomyces cerevisiae/interactome.txt", sep="\t")
interactome

,P1,P2
0,142,850399
1,142,850500
2,142,851572
3,142,853494
4,142,855159
...,...,...
229691,9164967,856237
229692,9164967,856459
229693,9164977,850883
229694,9164990,851529


In [33]:
sommets = {g for g in interactome["P1"]}.union({g for g in interactome["P2"]})
G = {str(g): set() for g in sommets}

for interaction in interactome.itertuples():
    G[str(interaction.P1)] = G[str(interaction.P1)].union({(str(interaction.P2), 1)})
    G[str(interaction.P2)] = G[str(interaction.P2)].union({(str(interaction.P1), 1)})

In [27]:
Louvain(G, 4.35)

KeyboardInterrupt: 